In [1]:
from glob import glob
import ctypes
import struct
import sys
from pathlib import Path

import numpy as np
import pandas

import vtu
PyVtu=vtu.PyVtu
import small_functions
import ts_auto_wrapper as wrap

In [2]:
ts=wrap.TSWrapper()

In [3]:
example_vtu_file = '/mnt/c/Users/yoavr/Desktop/timestep_000087.vtu'
vesicle = ts.parseDump(example_vtu_file)
v = PyVtu(example_vtu_file)

In [4]:
# import cffi # :(

In [5]:
vesicle.contents.cm

In [6]:
base_tape=ts.parsetape(str(ts.path_to_trisurf/'src/tape'))

In [7]:
tstape = ts.pretty_text(base_tape)
with open(ts.path_to_trisurf/'src/tape','r') as f:
    extape = small_functions.get_tape_options(f.read())
all_lines = {}
for line1 in tstape.splitlines():
    first=line1.split()[0]
    if first in extape:
        all_lines[first] = [line1, extape[first]]
    else:
        all_lines[first] = [line1, None]
pandas.DataFrame(all_lines)

,<ts_auto_wrapper.LP_ts_tape,tape_text,R_nucleus,R_nucleusX,R_nucleusY,R_nucleusZ,xkA0,xkV0,V0,A0,...,area_switch,quiet,plane_confinement_switch,allow_center_mass_movement,force_balance_along_z_axis,adhesion_geometry,adhesion_model,bond_model,curvature_model,force_model
0,<ts_auto_wrapper.LP_ts_tape object at 0x7f7898...,tape_text <ctypes.LP_c_char object at 0x7f...,R_nucleus 0.0,R_nucleusX 0.0,R_nucleusY 0.0,R_nucleusZ 0.0,xkA0 1.0,xkV0 1.0,V0 0.0,A0 0.0,...,area_switch b'\x00',quiet b'\x00',plane_confinement_switch b'\x00',allow_center_mass_movement b'\x00',force_balance_along_z_axis b'\x00',adhesion_geometry b'\x00',adhesion_model b'\x00',bond_model b'\x00',curvature_model b'\x0f',force_model b'\x00'
1,None,None,0,0,0,0,1.0,1.0,0,0,...,0,false,0,0,0,0,0,0,15,0


In [8]:
np.array([vesicle.contents.vlist.contents.vtx[i].contents.c for i in range(50,100)])

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [9]:
vtx=vesicle.contents.vlist.contents.vtx[97]
vtx_old=ts.ts_vertex(x=0,y=0,z=0)

In [10]:
vtx.contents.z, v.pos[97,2]

(-0.8323908581740378, -0.8323908581740378)

In [11]:
ts.functions.vesicle_meancurvature(vesicle), v.mean_curvature.sum()

(193.32805182985842, 386.65610365971753)

In [12]:
ts.parseDump?

Signature:
ts.parseDump(
    *args,
    _base_c_function=<_FuncPtr object at 0x7f78981fcf40>,
    arg_types=[<class 'ctypes.LP_c_char'>],
    process_args=[<function TSWrapper.__init__.<locals>.process_str_to_char_p at 0x7f7904068a60>],
    process_out=[<function TSWrapper.__init__.<locals>.process_neutral at 0x7f7907791510>, None],
)
Docstring: Wrapper for parseDump with arguments , dumpfname: str or __str__. Return <class 'ts_auto_wrapper.LP_ts_vesicle'>
File:      /opt/workspace/msc_project/lab_note_and_py/trisurf-python/ts_auto_wrapper.py
Type:      function

In [13]:
ts.cdll.direct_force_energy.restype=ctypes.c_double
ts.cdll.direct_force_energy(vesicle, vtx, ctypes.pointer(vtx_old))

0.0

In [14]:
wrap.parse_type("int **func",ts.ts_types)

('int', {'func': ts_auto_wrapper.LP_LP_c_int})

In [15]:
ts.byte_to_int(vesicle.contents.tape.contents.bond_model)

0

In [16]:
ts.dec_funcs['next_idx']

('ts_idx', ctypes.c_int, 'ts_idx i, ts_idx max')

In [17]:
ts.functions._c_next_idx.restype

ctypes.c_int

In [18]:
ts.functions.prev_small(4,8)

3

In [19]:
ts.dec_funcs['parsetape']

('ts_tape', ts_auto_wrapper.LP_ts_tape, 'char *filename')

In [20]:
cluster = ts.functions.init_cluster_list()

In [21]:
import inspect

In [22]:
def get_default_args(func):
    signature = inspect.signature(func)
    return {
        k: v.default
        for k, v in signature.parameters.items()
        if v.default is not inspect.Parameter.empty
    }

In [23]:
df=get_default_args(ts.functions.clusterize_vesicle)
process_args =df['process_args']
func = df['_base_c_function']

In [24]:
func(vesicle,cluster)

b'\x00'

In [25]:
[f(x) for f,x in zip(process_args,(vesicle,cluster))]

In [26]:
ts.functions._c_clusterize_vesicle.argtype

[ts_auto_wrapper.LP_ts_vesicle, ts_auto_wrapper.LP_ts_cluster_list]

In [27]:
ts.functions.clusterize_vesicle(vesicle,cluster)

True

In [28]:
cluster.contents.cluster.contents.contents.vtx.contents.contents.c

1.0

In [29]:
ts.functions.clusterize_vesicle

<function ts_auto_wrapper.TSWrapper.__init__.<locals>.f(*args, _base_c_function=<_FuncPtr object at 0x7f7898255900>, arg_types=[<class 'ts_auto_wrapper.LP_ts_vesicle'>, <class 'ts_auto_wrapper.LP_ts_cluster_list'>], process_args=[<function TSWrapper.__init__.<locals>.process_neutral at 0x7f7907791510>, <function TSWrapper.__init__.<locals>.process_neutral at 0x7f7907791510>], process_out=[<function TSWrapper.__init__.<locals>.process_ts_bool_to_true_false at 0x7f78983dc9d0>, None, None])>

In [30]:
ts.functions.single_timestep(vesicle)

(True, 0.5868244323632875, 0.01311160857051487)

In [31]:
ts.functions.vesicle_meancurvature?

Signature:
ts.functions.vesicle_meancurvature(
    *args,
    _base_c_function=<_FuncPtr object at 0x7f78981feb00>,
    arg_types=[<class 'ts_auto_wrapper.LP_ts_vesicle'>],
    process_args=[<function TSWrapper.__init__.<locals>.process_neutral at 0x7f7907791510>],
    process_out=[<function TSWrapper.__init__.<locals>.process_neutral at 0x7f7907791510>, None],
)
Docstring: Wrapper for vesicle_meancurvature with arguments , vesicle: <class 'ts_auto_wrapper.LP_ts_vesicle'>. Return <class 'ctypes.c_double'>
File:      /opt/workspace/msc_project/lab_note_and_py/trisurf-python/ts_auto_wrapper.py
Type:      function

In [32]:
ts.functions.vesicle_meancurvature(vesicle)

192.85626016870233